In other words, agents that make use of LLMs can be powerful general problem solvers.

Although the tools they use are important, the driving force of many agent-based systems is the use of a framework called **Reasoning and Acting (ReAct1)**.

# The Driving Power Behind Agents: Step-by-step Reasoning (ReAct)

Since LLMs can only generate text, they would need to be instructed to use specific queries to trigger tools like a forecasting API.

ReAct merges these two concepts and allows reasoning to affect acting and actions to affect reasoning. In practice, the framework consists of iteratively following these three steps:

- Thought
- Action
- Observation

It goes like:
- the LLM is asked to create a “thought” about the input prompt
- based on the thought, an “action” is triggered (The action is generally an external tool, like a calculator or a search engine)
- after the results of the “action” are returned to the LLM it “observes” the output


## ReAct in LangChain

To illustrate how agents work in LangChain, we are going to build a pipeline that can search the web for answers and perform calculations with a calculator.

In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults
from langchain.agents import AgentExecutor, create_react_agent

load_dotenv()

True

In [2]:
# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [3]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [4]:
# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

In [5]:
# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [6]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [7]:
# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)

In [8]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [9]:
# What is the price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD."
    }
)



> Entering new AgentExecutor chain...
I should use a web search engine to find the current price of a MacBook Pro in USD and then use a calculator to convert it to EUR.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"[snippet: We found all the best discounts on Apple's MacBook Air and MacBook Pro laptops. ... List price Best price (current) Best price (all-time) M2 MacBook Air (13-inch) $999: $749: $700:, title: Best MacBook Deals: Enjoy Discounts on Apple's Laptops Right Here, link: https://www.cnet.com/deals/best-macbook-deals/], [snippet: The 14-inch MacBook Pro M4 is one of the best MacBooks we've tested and comes with our highest recommendation, especially at this lower price. MacBook Pro 14" (M4) was $1,599 now $1,399 @ Amazon, title: The new M4 MacBook Pro 14-inch is already $200 off before Black Friday ..., link: https://www.tomsguide.com/computing/macbooks/hurry-the-new-m4-macbook-pro-14-inch-just-dropped-to-lowest-price-yet-at-amazon], [snippet: For the 

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD.',
 'output': 'The current price of a MacBook Pro in USD is $1399. If the exchange rate is 0.85 EUR for 1 USD, it would cost approximately 1189.15 EUR.'}